## Experiments: Sensitive Analysis Hyperparameter RSAST:


It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 30 ,50 and 100 for the selected dataset.

In [105]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST',
 'c:\\ProgramData\\Anaconda3\\python39.zip',
 'c:\\ProgramData\\Anaconda3\\DLLs',
 'c:\\ProgramData\\Anaconda3\\lib',
 'c:\\ProgramData\\Anaconda3',
 '',
 'C:\\Users\\Nicolas R\\AppData\\Roaming\\Python\\Python39\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin',
 'c:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\Nicolas R\\.ipython',
 'c:\\Users\\Nicolas R\\random_sast/sast',
 'c:\\Users\\Nicolas R\\random_sast\\sast',
 'c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST/sast',
 'c:\\Users\\Nicolas R\\random_sast\\ExperimentationRSAST\\sast',
 'c:\\Users\\Nicolas R\\random_sast\\cd_diagram',
 'c:\\Users\\Nicolas R\\random_sast/cd_diagram',
 'c:\\Users\\Nicolas R\\random_sast\\Experimentation

In [106]:
from sast import *
import pandas as pd
import researchpy
import math
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns
from cd_function import *
from utils_sast import delete_files_in_directory

### Reading Datasets

In [107]:
# Set directory where the RSAST result csv files are located

directories=[]

default_split_acfandpacf_k_1_10_sast=True
default_split_acfandpacf_k_1_10_notsast=True
default_split_all_k_1_10=True
default_split_all_acfandpacf_k_30_50=True
default_split_maxpacf_none_k_30_50=True

default_split_all_acfandpacf_maxpacf_none_maxacf_k_1_50_p_1_100=False

resampling_acfandpacf_k_1_10_sast=False

if default_split_acfandpacf_k_1_10_sast:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST')
if default_split_acfandpacf_k_1_10_notsast:
    directories.append(os.getcwd()+'/results_rsast/Server17_not_used_sast')
if default_split_all_k_1_10:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST_All')   
if default_split_all_acfandpacf_k_30_50:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST_Extra_1')    
if default_split_maxpacf_none_k_30_50:
    directories.append(os.getcwd()+'/results_rsast/Server17_Comparison_RSAST_Extra_2')    
if resampling_acfandpacf_k_1_10_sast:
    directories.append(os.getcwd()+'/results_rsast/Server17_Resampling_Comparison_RSAST')     
if default_split_all_acfandpacf_maxpacf_none_maxacf_k_1_50_p_1_100:
    directories.append(os.getcwd()+'/results_rsast/Server16_Hyperparameter_Tunning')
    directories.append(os.getcwd()+'/results_rsast/Server17_Hyperparameter_Tunning')


# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            df['directory']=directory
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)


C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\1086993268.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\1086993268.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\1086993268.py:47: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\1086993268.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.repl

In [108]:
print("Total DS:" + str(len(df_result['dataset_name'].unique())))

Total DS:128


In [109]:
df_print=pd.pivot_table(df_result, index="dataset_name", values="accuracy", columns="classifier_name").reset_index().round(2)

if not resampling_acfandpacf_k_1_10_sast:
       df_print.to_csv("results_default_split.csv")
else: 
       df_print.to_csv("results_10resampling.csv")
df_print

classifier_name,dataset_name,ACF&PACF: n_random_points=10 nb_inst_per_class=1,ACF&PACF: n_random_points=10 nb_inst_per_class=10,ACF&PACF: n_random_points=10 nb_inst_per_class=30,ACF&PACF: n_random_points=10 nb_inst_per_class=50,ACF&PACF: n_random_points=30 nb_inst_per_class=1,ACF&PACF: n_random_points=30 nb_inst_per_class=10,ACF&PACF: n_random_points=30 nb_inst_per_class=30,ACF&PACF: n_random_points=30 nb_inst_per_class=50,Max PACF: n_random_points=10 nb_inst_per_class=1,...,None: n_random_points=30 nb_inst_per_class=30,None: n_random_points=30 nb_inst_per_class=50,all: n_random_points=10 nb_inst_per_class=1,all: n_random_points=10 nb_inst_per_class=10,all: n_random_points=10 nb_inst_per_class=30,all: n_random_points=10 nb_inst_per_class=50,all: n_random_points=30 nb_inst_per_class=1,all: n_random_points=30 nb_inst_per_class=10,all: n_random_points=30 nb_inst_per_class=30,all: n_random_points=30 nb_inst_per_class=50
0,ACSF1,0.40,0.35,NaN,NaN,0.37,0.32,NaN,NaN,0.43,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Adiac,0.63,0.58,0.58,0.58,0.61,0.57,0.57,0.57,0.70,...,0.63,0.64,0.60,0.55,0.56,0.55,0.56,0.55,0.55,0.55
2,AllGestureWiimoteX,0.47,0.44,NaN,NaN,0.43,0.53,NaN,NaN,0.50,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AllGestureWiimoteY,0.51,0.49,NaN,NaN,0.49,0.58,NaN,NaN,0.52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AllGestureWiimoteZ,0.46,0.38,NaN,NaN,0.38,0.46,NaN,NaN,0.44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,Wine,0.85,0.78,0.76,0.79,0.78,0.78,0.77,0.79,0.69,...,0.87,0.86,0.82,0.88,0.87,0.88,0.84,0.89,0.89,0.89
124,WordSynonyms,0.66,0.68,0.69,0.68,0.67,0.68,0.69,0.68,0.62,...,0.68,0.67,0.67,0.69,0.69,0.69,0.68,0.70,0.70,0.70
125,Worms,0.46,0.48,NaN,NaN,0.41,0.51,NaN,NaN,0.49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126,WormsTwoClass,0.57,0.63,NaN,NaN,0.60,0.65,NaN,NaN,0.60,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
pv_all=pd.pivot_table(df_result,index='dataset_name',values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
ds_complete
df_result=df_result[df_result.dataset_name.isin(ds_complete)]

In [111]:
print("Complete (max runs) per DS:" + str(len(df_result['dataset_name'].unique())))

Complete (max runs) per DS:72


In [112]:
pv_all=pd.pivot_table(df_result,index='len_method',values='accuracy',aggfunc="count")
len_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().len_method.unique()
len_complete
df_result=df_result[df_result.len_method.isin(len_complete)]

In [113]:
print("Complete (max runs) per Len Method:" + str(len(df_result['dataset_name'].unique())))

Complete (max runs) per Len Method:72


In [114]:
#get ds tested overall
print("Used Datasets:")
pd.pivot_table(df_result, index="dataset_name", values="accuracy", aggfunc="count").reset_index()["dataset_name"].unique()

Used Datasets:


array(['Adiac', 'ArrowHead', 'BME', 'Beef', 'BeetleFly', 'BirdChicken',
       'CBF', 'Car', 'Chinatown', 'ChlorineConcentration', 'Coffee',
       'CricketX', 'CricketY', 'CricketZ', 'Crop', 'DiatomSizeReduction',
       'DistalPhalanxOutlineAgeGroup', 'DistalPhalanxOutlineCorrect',
       'DistalPhalanxTW', 'DodgerLoopDay', 'DodgerLoopGame',
       'DodgerLoopWeekend', 'ECG200', 'ECG5000', 'ECGFiveDays',
       'Earthquakes', 'ElectricDevices', 'FaceAll', 'FaceFour',
       'FacesUCR', 'FiftyWords', 'FreezerRegularTrain',
       'FreezerSmallTrain', 'Fungi', 'GunPoint', 'GunPointAgeSpan',
       'GunPointMaleVersusFemale', 'GunPointOldVersusYoung', 'Ham',
       'Herring', 'InsectWingbeatSound', 'ItalyPowerDemand', 'Meat',
       'MedicalImages', 'MelbournePedestrian',
       'MiddlePhalanxOutlineAgeGroup', 'MiddlePhalanxOutlineCorrect',
       'MiddlePhalanxTW', 'MoteStrain', 'PhalangesOutlinesCorrect',
       'Plane', 'PowerCons', 'ProximalPhalanxOutlineAgeGroup',
       'ProximalP

In [115]:
#get ds tested overall
print("Total Param Combination:" + str(len(df_result['classifier_name'].unique())))

Total Param Combination:32


In [116]:
#get len methods of generated datasets
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=30',
       'all: n_random_points=

In [117]:
#get columns of generated datasets
df_result.columns.unique()

Index(['Unnamed: 0', 'accuracy', 'time', 'cweights_time', 'fsubsequence_time',
       'tdataset_time', 'tclassifier_time', 'dataset_name', 'classifier_name',
       'rpoint', 'nb_per_class', 'method', 'len_method', 'directory'],
      dtype='object')

### Hyperparameter Tunning RSAST 

#### Accuracy: Subsequence Lenght Method 

In [118]:
# Summary statistics for a Series (single variable)
summ_ds=df_result
summ_ds=researchpy.summary_cont(summ_ds.groupby(['dataset_name'])['accuracy'], conf = 0.95)
summ_ds

,N,Mean,SD,SE,95% Conf.,Interval
dataset_name,,,,,,
Adiac,160,0.5743,0.0604,0.0048,0.5649,0.5837
ArrowHead,160,0.7290,0.0477,0.0038,0.7215,0.7364
BME,160,0.8341,0.0456,0.0036,0.8270,0.8412
Beef,160,0.6965,0.0988,0.0078,0.6810,0.7119
BeetleFly,160,0.7769,0.0920,0.0073,0.7625,0.7912
...,...,...,...,...,...,...
TwoPatterns,160,0.9459,0.0908,0.0072,0.9317,0.9601
UMD,160,0.9119,0.0930,0.0074,0.8974,0.9265
Wafer,160,0.9960,0.0058,0.0005,0.9951,0.9969


In [119]:
stats = df_result.groupby(['len_method'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))

stats['ci95_hi'] = ci95_hi
stats['ci95_lo'] = ci95_lo
print(stats.head(10))

                mean  count       std   ci95_hi   ci95_lo
len_method                                               
ACF&PACF    0.834056   2880  0.139293  0.839144  0.828969
Max PACF    0.802146   2880  0.147317  0.807527  0.796766
None        0.806014   2880  0.144266  0.811283  0.800745
all         0.835613   2880  0.142135  0.840804  0.830422


#### Generate Boxplot Tunning Hyperparameter

In [120]:
df_result.nb_per_class.unique()

array(['1', '10', '30', '50'], dtype=object)

In [121]:
df_result.len_method.unique()

array(['ACF&PACF', 'Max PACF', 'None', 'all'], dtype=object)

In [122]:
#generate hyperparameter tuning boxplots
delete_files_in_directory("images_boxplot_acc/")
for k, ints in enumerate(df_result.nb_per_class.unique()):
    for len_m in df_result.len_method.unique():
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        # Plot
        fig, ax = plt.subplots()

        sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
        
        max_bx=max(df_to_cd.accuracy)
        min_bx=min(df_to_cd.accuracy)
        
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )

        # save plot
        plt.savefig('images_boxplot_acc/boxplot_acc'+len_m+ints+'.png')


All files deleted successfully.


In [123]:
df_result.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'None: n_random_points=10 nb_inst_per_class=1',
       'None: n_random_points=10 nb_inst_per_class=10',
       'None: n_random_points=30 nb_inst_per_class=1',
       'None: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=1',
       'all: n_random_points=10 nb_inst_per_class=10',
       'all: n_random_points=30 nb_inst_per_class=1',
       'all: n_random_points=30 nb_inst_per_class=10',
       'all: n_random_points=10 nb_inst_per_class=30',
       'all: n_random_points=

In [124]:
delete_files_in_directory("images_boxplot_acc_per_ds/")
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():
            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]
            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.accuracy)
            min_bx=min(df_to_cd.accuracy)
            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]    
            
            # Plot
            fig, ax = plt.subplots()
            sns.boxplot(data=df_to_cd, x='rpoint', y='accuracy', palette="Blues")
            ax.set_ylim(min_bx,max_bx)

            # Axis details
            ax.set(xlabel='number of random points', ylabel='accuracy', title='Accuracy Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )

            # save plot
            plt.savefig('images_boxplot_acc_per_ds/boxplot_acc'+len_m+ints+'_'+ds+'.png')

All files deleted successfully.


C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\1503050523.py:17: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax.set_ylim(min_bx,max_bx)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\1503050523.py:17: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax.set_ylim(min_bx,max_bx)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\1503050523.py:17: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax.set_ylim(min_bx,max_bx)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\1503050523.py:17: UserWarning: Attempting to set identical bottom == top == 1.0 results in singular transformations; automatically expanding.
  ax.set_ylim(min_bx,max_bx)
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\1503050523.py:17: UserWarning: Attempting to set id

In [125]:
delete_files_in_directory("images_boxplot_time/")
#generate hyperparameter tuning boxplots
for k, ints in enumerate(df_result.nb_per_class.unique()):
    for len_m in df_result.len_method.unique():
                
        df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':.*nb_inst_per_class='+str(ints)+'$')]
        df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
        df_to_cd["time"]=df_to_cd["time"]/60

        # Plot
        fig, ax = plt.subplots()
        sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")      
        max_bx=max(df_to_cd.time)
        min_bx=min(df_to_cd.time)
        ax.set_ylim(min_bx,max_bx)
        # Axis details
        ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
        #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )

        # save plot
        plt.savefig('images_boxplot_time/boxplot_time'+len_m+ints+'.png')


All files deleted successfully.


C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\191012824.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\191012824.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_cd["time"]=df_to_cd["time"]/60
C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\191012824.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [126]:
delete_files_in_directory("images_boxplot_time_per_ds/")
#generate hyperparameter tuning boxplots
for ds in df_result.dataset_name.unique():
    for k, ints in enumerate(df_result.nb_per_class.unique()):
        for len_m in df_result.len_method.unique():      

            df_to_cd=df_result[df_result.classifier_name.str.contains(r'^'+len_m+':')]
            df_to_cd=df_to_cd[df_to_cd.dataset_name==ds]

            df_to_cd["time"]=df_to_cd['cweights_time']+df_to_cd['fsubsequence_time']+df_to_cd['tdataset_time']
            df_to_cd["time"]=df_to_cd["time"]/60
            max_bx=max(df_to_cd.time)
            min_bx=min(df_to_cd.time)

            df_to_cd=df_to_cd[df_to_cd.classifier_name.str.contains(r':.*nb_inst_per_class='+str(ints)+'$')]

            # Plot
            fig, ax = plt.subplots()
                       
            #order=list(df_to_cd)
            sns.boxplot(data=df_to_cd, x='rpoint', y='time', palette="Blues")

            ax.set_ylim(min_bx,max_bx)
            # Axis details
            ax.set(xlabel='number of random points', ylabel='time', title='Time Boxplot for Len method:'+len_m+' and N° of Instances:'+ints)
            #plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )

            # save plot
            plt.savefig('images_boxplot_time_per_ds/boxplot_time'+len_m+ints+'_'+ds+'.png')


All files deleted successfully.


#### Generate CD Diagram Tunning Hyperparameter

In [127]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_perf.classifier_name.unique()

array(['ACF&PACF: n_random_points=10 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=30',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=50',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=1',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=10',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=30',
       'ACF&PACF: n_random_points=30 nb_inst_per_class=50',
       'Max PACF: n_random_points=10 nb_inst_per_class=1',
       'Max PACF: n_random_points=10 nb_inst_per_class=10',
       'Max PACF: n_random_points=10 nb_inst_per_class=30',
       'Max PACF: n_random_points=10 nb_inst_per_class=50',
       'Max PACF: n_random_points=30 nb_inst_per_class=1',
       'Max PACF: n_random_points=30 nb_inst_per_class=10',
       'Max PACF: n_random_points=30 nb_inst_per_class=30',
       'Max PACF: n_random_points=30 nb_inst_per_class=50',
       'None: n_random_points=10 nb_inst_per

In [128]:
delete_files_in_directory("images_cd_diagram/")
#generate hyperparameter tuning cd diagrams

best_comb_by_method=[]
best=""
for len_m in df_perf.len_method.unique():
   df_to_cd=df_perf[df_perf.classifier_name.str.contains(r'^'+len_m+':.*$')]
   draw_cd_diagram(df_to_cd, labels=True, title=len_m +" comparison", fname='images_cd_diagram/cd-diagram_'+len_m+'.png')
   _, average_ranks, _ = wilcoxon_holm(df_perf=df_to_cd)
   min_rank= min(average_ranks)
   average_ranks=pd.DataFrame(average_ranks)
   best=average_ranks[average_ranks[0]==min_rank][0].index
   best_comb_by_method.append(best[0])



All files deleted successfully.
{0: 'ACF&PACF: n_random_points=10 nb_inst_per_class=1', 1: 'ACF&PACF: n_random_points=30 nb_inst_per_class=1', 2: 'ACF&PACF: n_random_points=10 nb_inst_per_class=10', 3: 'ACF&PACF: n_random_points=30 nb_inst_per_class=10', 4: 'ACF&PACF: n_random_points=10 nb_inst_per_class=30', 5: 'ACF&PACF: n_random_points=30 nb_inst_per_class=50', 6: 'ACF&PACF: n_random_points=10 nb_inst_per_class=50', 7: 'ACF&PACF: n_random_points=30 nb_inst_per_class=30'}
[0, 1]
[2, 3, 4, 5, 6, 7]
{0: 'Max PACF: n_random_points=10 nb_inst_per_class=1', 1: 'Max PACF: n_random_points=30 nb_inst_per_class=1', 2: 'Max PACF: n_random_points=10 nb_inst_per_class=10', 3: 'Max PACF: n_random_points=10 nb_inst_per_class=30', 4: 'Max PACF: n_random_points=30 nb_inst_per_class=10', 5: 'Max PACF: n_random_points=10 nb_inst_per_class=50', 6: 'Max PACF: n_random_points=30 nb_inst_per_class=30', 7: 'Max PACF: n_random_points=30 nb_inst_per_class=50'}
[0, 1]
[2, 3, 4, 5]
[6, 7]
{0: 'None: n_random_p

In [129]:
best_comb_by_method

['ACF&PACF: n_random_points=30 nb_inst_per_class=30',
 'Max PACF: n_random_points=30 nb_inst_per_class=50',
 'None: n_random_points=30 nb_inst_per_class=50',
 'all: n_random_points=30 nb_inst_per_class=50']

In [130]:
df_perf=pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name','len_method'], aggfunc='mean')
df_perf=df_perf.reset_index()
df_to_cd=df_perf[df_perf.classifier_name.isin(best_comb_by_method)]

In [131]:
draw_cd_diagram(df_to_cd, labels=True, title="Best combination comparison", fname='images_cd_diagram/cd-diagram_best_com.png')


{0: 'Max PACF: n_random_points=30 nb_inst_per_class=50', 1: 'None: n_random_points=30 nb_inst_per_class=50', 2: 'ACF&PACF: n_random_points=30 nb_inst_per_class=30', 3: 'all: n_random_points=30 nb_inst_per_class=50'}
[1, 0]
[1, 2]
[3, 2]


In [132]:
df_perf_all=df_perf[['dataset_name','len_method','accuracy']]
df_perf_all.rename(columns = {'len_method':'classifier_name'}, inplace = True)
df_perf_all=pd.pivot_table(df_perf_all, index=["dataset_name","classifier_name"]).reset_index()

C:\Users\Nicolas R\AppData\Local\Temp\ipykernel_1416\293824979.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_perf_all.rename(columns = {'len_method':'classifier_name'}, inplace = True)


In [133]:
draw_cd_diagram(df_perf_all, labels=True, fname='images_cd_diagram/cd-diagram_all_methods.png')

{0: 'Max PACF', 1: 'None', 2: 'ACF&PACF', 3: 'all'}
[0, 1]
[2, 3]


#### Focus on most accurate lenght method

##### Overall Accuracy

In [134]:
# highest accurate hyperparameters
best_comb_by_method

['ACF&PACF: n_random_points=30 nb_inst_per_class=30',
 'Max PACF: n_random_points=30 nb_inst_per_class=50',
 'None: n_random_points=30 nb_inst_per_class=50',
 'all: n_random_points=30 nb_inst_per_class=50']

In [135]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result
#df_result_acc=df_result_acc[df_result_acc["dataset_name"]=="Fungi"]

pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.788595,0.824341,0.828194,0.830629,NaN,NaN,NaN
30,NaN,0.790650,0.827105,0.833057,0.833087,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [136]:
# create a pivot table with the variatioon of score by hyperparameter
pivot = pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.020597,0.019937,0.020079,0.019874,NaN,NaN,NaN
30,NaN,0.023089,0.019885,0.019827,0.020480,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Overall time complexity

In [137]:
df_result_acc.time.sum()/(60*60*24)

13.874055321249697

In [138]:
# create a pivot table with the mean of time spent in minutes
pivot=pd.pivot_table(df_result_acc, values=['time','cweights_time','fsubsequence_time','tdataset_time','tclassifier_time'], index=['dataset_name'], aggfunc='sum')/60
pivot = pivot.reindex(columns=['cweights_time','fsubsequence_time','tdataset_time','tclassifier_time','time'])
pivot

,cweights_time,fsubsequence_time,tdataset_time,tclassifier_time,time
dataset_name,,,,,
Adiac,0.867596,25.712476,901.273678,14.899561,943.014502
ArrowHead,0.167042,7.224477,15.629679,0.204263,23.254475
BME,0.091900,1.265460,3.307109,0.061809,4.738975
Beef,0.376718,55.387486,62.662670,0.118091,118.597549
BeetleFly,0.264592,50.731055,18.303402,0.052687,69.386501
...,...,...,...,...,...
TwoPatterns,0.210125,4.542338,131.874155,12.871888,149.594443
UMD,0.105202,1.864404,5.475742,0.157836,7.622142
Wafer,0.183655,3.120209,60.313858,7.008007,70.691299


In [139]:
# create a pivot table with the mean of time spent in minutes by random points and instances per class
pivot = pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')/60
pivot = pivot.reindex(columns=["half_instance","1","10","30","50","100","1000","10000"], index=["half_len","1","10","30","50","100","1000","10000"])
pivot

nb_per_class,half_instance,1,10,30,50,100,1000,10000
rpoint,,,,,,,,
half_len,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,0.368510,2.938477,0.277818,0.321301,NaN,NaN,NaN
30,NaN,0.928246,8.023154,0.449178,0.567370,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Calculate weights time complexity

In [140]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50
rpoint,,,,
10,0.104732,0.103949,0.087187,0.084954
30,0.103705,0.104504,0.086132,0.087653


##### Finding subsequences time complexity

In [141]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50
rpoint,,,,
10,1.333338,11.307434,8.416147,9.469907
30,1.424809,11.551143,8.711501,10.123382


##### Transform Dataset time complexity

In [142]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50
rpoint,,,,
10,17.689851,160.498545,3.993123,5.018143
30,51.122587,464.089302,11.268979,14.205263


##### Classifier time complexity

In [143]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,30,50
rpoint,,,,
10,2.967764,4.387551,4.125909,4.673310
30,3.033969,5.621210,6.834368,9.554805
